In [ ]:
import os
import json
import numpy as np
import location_finding
import hyperbolic_temporal_discount
import death_process
import irt
import survival_analysis
import peregrines
import dugongs
import lotka_volterra
import moral_machines
import emotion
import rat_tumor

In [ ]:
results_dir = "./results"
exp = "discovery"
env = "morals"
goal = "direct_naive"
model = "gpt-4o"
seeds = [1,2,3,4,5] 
box = False 
if box:
    model += "-boxloop"

In [ ]:
nametoenv = {    
    "location_finding": location_finding.Signal,
    "hyperbolic_temporal_discount": hyperbolic_temporal_discount.TemporalDiscount,
    "death_process": death_process.DeathProcess,
    "irt": irt.IRT,
    "survival": survival_analysis.SurvivalAnalysis,
    "dugongs": dugongs.Dugongs,
    "peregrines": peregrines.Peregrines,
    "morals": moral_machines.MoralMachine,
    "emotion": emotion.EmotionFromOutcome,
    "lotka_volterra": lotka_volterra.LotkaVolterra,
    "rat_tumor": rat_tumor.RatTumorModel
}
nameenvtogoal = {
    ("hyperbolic_temporal_discount", "direct"): hyperbolic_temporal_discount.DirectGoal,
    ("hyperbolic_temporal_discount", "discount"): hyperbolic_temporal_discount.DiscountGoal,
    ("hyperbolic_temporal_discount", "direct_naive"): hyperbolic_temporal_discount.DirectGoalNaive,
    ("location_finding", "direct"): location_finding.DirectGoal,
    ("location_finding", "source"): location_finding.SourceGoal,
    ("location_finding", "direct_naive"): location_finding.DirectGoalNaive,
    ("death_process", "direct"): death_process.DirectDeath,
    ("death_process", "direct_naive"): death_process.DirectDeathNaive,
    ("death_process", "infection"): death_process.InfectionRate,
    ("irt", "direct"): irt.DirectCorrectness,
    ("irt", "direct_naive"): irt.DirectCorrectnessNaive,
    ("irt", "best_student"): irt.BestStudent,
    ("irt", "difficult_question"): irt.DifficultQuestion,
    ("irt", "discriminate_question"): irt.DiscriminatingQuestion,
    ("survival", "direct"): survival_analysis.DirectGoal,
    ("survival", "direct_naive"): survival_analysis.DirectGoalNaive,
    ("dugongs", "direct"): dugongs.DirectGoal,
    ("dugongs", "direct_naive"): dugongs.DirectGoalNaive,
    ("peregrines", "direct"): peregrines.DirectGoal,
    ("peregrines", "direct_naive"): peregrines.DirectGoalNaive,
    ("emotion", "direct"): emotion.DirectEmotionPrediction,
    ("emotion", "direct_naive"): emotion.DirectEmotionNaive,
    ("morals", "direct"): moral_machines.DirectPrediction,
    ("morals", "direct_naive"): moral_machines.DirectPredictionNaive,
    ("lotka_volterra", "direct"): lotka_volterra.DirectGoal,
    ("lotka_volterra", "direct_naive"): lotka_volterra.DirectGoalNaive,
    ("rat_tumor", "direct"): rat_tumor.DirectGoal,
    ("rat_tumor", "direct_naive"): rat_tumor.DirectGoalNaive
}

In [ ]:
environ = nametoenv[env]()
goaln = nameenvtogoal[env, goal](environ)
norm_mu = goaln.norm_mu
norm_sigma = goaln.norm_sigma



In [ ]:
files = [os.path.join(results_dir, env, f"{goal}_{model}_{exp}_true_{seed}.json") for seed in seeds]
files_no_prior = [os.path.join(results_dir, env, f"{goal}_{model}_{exp}_false_{seed}.json") for seed in seeds]

In [ ]:
data = []
for file in files:
    print(file)
    with open(file, "r") as f:
        data.append(json.load(f))
data_no_prior = []
for file in files_no_prior:
    print(file)
    with open(file, "r") as f:
        data_no_prior.append(json.load(f))

In [ ]:
print(len(data[0]['data']['queries']))
# (mean, std), questions, gts, predictions

In [ ]:
errs, stds = [], []
for d in data:
    errs.append([])
    stds.append([])
    for r in d['data']['results']: 
        errs[-1].append(r[0][0])
        stds[-1].append(r[0][1])
errs_no_prior, stds_no_prior = [], []
for d in data_no_prior:
    errs_no_prior.append([])
    stds_no_prior.append([])
    for r in d['data']['results']: 
        errs_no_prior[-1].append(r[0][0])
        stds_no_prior[-1].append(r[0][1])

errs = np.array(errs)
stds = np.array(stds)
errs_no_prior = np.array(errs_no_prior)
stds_no_prior = np.array(stds_no_prior)

if env == "location_finding" and "direct" in goal:
    # clip
    errs = np.clip(errs, 0, 10000)
    errs_no_prior = np.clip(errs_no_prior, 0, 10000)


# normalize
errs = (errs - norm_mu) / norm_sigma
errs_no_prior = (errs_no_prior - norm_mu) / norm_sigma

if env == "hyperbolic_temporal_discount" and "direct" in goal:
    errs = 1 - errs
    errs_no_prior = 1 - errs_no_prior
# elif env == "peregrines":
#     errs = errs ** 0.5
#     errs_no_prior = errs_no_prior ** 0.5
elif env == "survival":
    errs = 1 - errs
    errs_no_prior = 1 - errs_no_prior
if env == "morals":
    print("removing")
    errs = 1 - errs

mean_score = np.mean(errs, axis=0)
ci_95 = 1.96 * np.std(errs, axis=0) / np.sqrt(len(errs))
mean_score_no_prior = np.mean(errs_no_prior, axis=0)
ci_95_no_prior = 1.96 * np.std(errs_no_prior, axis=0) / np.sqrt(len(errs_no_prior))



print(mean_score.shape)

In [ ]:
print(mean_score, ci_95, mean_score_no_prior, ci_95_no_prior)